In [60]:
torch.tensor([[1,2]]).matmul(torch.tensor([[1],[2]]))

tensor([[5]])

In [1]:
import os
import mmcv
import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

/opt/conda/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [45]:
# epoch 지정해주기
epoch = 'epoch_70' # weight 이름
# work dir 설정해주기
work_dir = '/opt/ml/input/mmsegmentation/work_dirs/base_split1_pseudo'

# config file 들고오기
# config file은 수정해서 사용
cfg = Config.fromfile('/opt/ml/input/mmsegmentation/configs/_swinL_/_base_/base_split1_pseudo.py')
root='/opt/ml/input/data/mmseg/images/test'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [46]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2023-01-05 08:37:02,503 - mmseg - INFO - Loaded 819 images


In [47]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/input/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:276: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/input/mmsegmentation/work_dirs/base_split1_pseudo/epoch_70.pth


In [48]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.6 task/s, elapsed: 1479s, ETA:     0s

In [49]:
# np.array(output).shape

In [50]:
# 여기서 pickle 파일 생성하면 될듯

In [51]:
# submission 양식에 맞게 output 후처리
input_size = 512
output_size = 256

submission = pd.read_csv("/opt/ml/input/code/submission/sample_submission.csv", index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")

with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# PredictionString 대입
for image_id, predict in enumerate(output):

    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]

    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    mask = predict.reshape((1, output_size, input_size//output_size, output_size, input_size//output_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)
    string = oms.flatten()
    submission = pd.concat([submission, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}])]
                                   , ignore_index=True)


submission.to_csv(os.path.join(cfg.work_dir, f'swinL_uper_defaultaug_split1_pseudo_submission_{epoch}.csv'), index=False)

In [52]:
submission.head()

,image_id,PredictionString
0,batch_01_vt/0021.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,batch_01_vt/0028.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,batch_01_vt/0031.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,batch_01_vt/0032.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,batch_01_vt/0076.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
